### 数据预处理

#### 1. 根据语料分别构建源数据词表 和 目标数据词表

In [ ]:
import sys 
sys.path.append("..")
from Utils.Dictionary import Dictionary


data_path = "../Data/Dataset/data.csv"
source_dic_path = "../Data/source_vocal.pkl"
target_dic_path = "../Data/target_vocal.pkl"


# 生成 源数据字典 and 目标数据字典
source_dic = Dictionary()
source_dic.load_from_data(data_path, "raw_data")
source_dic.dump_to_file(source_dic_path)

target_dic = Dictionary()
target_dic.load_from_data(data_path, "raw_data")
target_dic.dump_to_file(target_dic_path)
target_dic.word2idx

#### 2. 数据集的划分（如果已经划分好就不必划分）

In [ ]:
import pandas as pd

# 文件路径
data_path = "../Data/Dataset/data.csv"
train_data_path = "../Data/Dataset/train_data_1.csv"
test_data_path = "../Data/Dataset/test_data_1.csv"

# 数据集划分
split_ratio = 0.6
dataset = pd.read_csv(data_path, index_col=0)
train_set = dataset.sample(frac = split_ratio, random_state=0, axis=0)
test_set = dataset[~dataset.index.isin(train_set.index)]

# 数据集保存
train_set.to_csv(train_data_path)
test_set.to_csv(test_data_path)

#### 数据增强

In [ ]:
import random
from Utils.VariantNeedleman_Wunsch import VariantNW
import pandas as pd
import pickle
import json


# 寻找可疑变异
def findVariant(seq_1, seq_2):
    variant_dic = {}
    for variant, right in zip(seq_1, seq_2):
        
        if isVariant(variant, right):
            if right not in variant_dic.keys():
                value = [variant]
                variant_dic[right] = value
            else:
                value = variant_dic[right]
                if variant not in value:
                    value.append(variant)
                variant_dic[right] = value

    return variant_dic

def addVariant(dic1, dic2):
    for k,v in dic2.items():
        if k in dic1.keys():
            for i in v:
                if i not in dic1[k]:
                    dic1[k].append(i)
        else:
            dic1[k] = v
    return dic1

def isVariant(char1, char2):
    if char1 == char2 or char1 == '[GAP]' or char2 == '[GAP]':
        return False
    else:
        return True

# 筛选变体字典
def filterVariant(variant_dic, remove_AandD = True, remove_single = True, remove_ratio = 0.5):
    variant_dic_2 = variant_dic.copy()

    for key, value in variant_dic.items():
        delete_key = False
        
        if (key.encode("utf-8").isalpha() or key.isdigit()) and remove_AandD:
            delete_key = True

        if len(value) == 1 and remove_single:
            delete_key = True
        
        if delete_key:
            del variant_dic_2[key]
        else:
            for val in value:
                variantNW.set_seqs(key, val)
                variantNW.propagate()
                aligned_seq1, aligned_seq2 = variantNW.traceback()
                score = variantNW.get_aligned_seq_score(aligned_seq1, aligned_seq2).pop()
                if score < remove_ratio:
                    variant_dic_2[key].remove(val)
                
    return variant_dic_2

data = pd.read_csv("../Data/Dataset/train_data.csv", index_col=0).reset_index(drop=True)
variantNW = VariantNW()

variant_dic = {}
for row in data.iterrows():
    seq1 = row[1][0]
    seq2 = row[1][1]
    variantNW.set_seqs(seq1, seq2)
    variantNW.propagate()
    aligned_seq1, aligned_seq2 = variantNW.traceback()
    dic = findVariant(aligned_seq1, aligned_seq2)
    variant_dic = addVariant(variant_dic, dic)

variant_dic = filterVariant(variant_dic)
variant_dic

In [ ]:
variant_dic = filterVariant(variant_dic)
variant_dic

In [ ]:
# 筛选变体字典
def filterVariant(variant_dic, remove_AandD = True, remove_single = True, remove_ratio = 0.5):
    variant_dic_2 = variant_dic.copy()

    for key, value in variant_dic.items():
        delete_key = False
        
        if (key.encode("utf-8").isalpha() or key.isdigit()) and remove_AandD:
            delete_key = True

        if len(value) == 1 and remove_single:
            delete_key = True
        
        if delete_key:
            del variant_dic_2[key]
        else:
            for val in value:
                variantNW.set_seqs(key, val)
                variantNW.propagate()
                aligned_seq1, aligned_seq2 = variantNW.traceback()
                score = variantNW.get_aligned_seq_score(aligned_seq1, aligned_seq2).pop()
                if score < remove_ratio:
                    variant_dic_2[key].remove(val)
                
    return variant_dic_2
variant_dic = filterVariant(variant_dic)
variant_dic
for k,v in variant_dic.items():
    if len(v) == 1:
        print(k)

In [ ]:
variant_dic = filterVariant(variant_dic)
variant_dic
with open("../Data/variant_dic_remove_re.json", "w", encoding='UTF-8') as f:
    json_str = json.dumps(variant_dic, indent=4, ensure_ascii=False)
    f.write(json_str)

In [ ]:
import random
from Utils.VariantNeedleman_Wunsch import VariantNW
import pandas as pd
import pickle
import json

# 判断是否是变异
def isVariant(char1, char2):
    if char1 == char2 or char1 == '[GAP]' or char2 == '[GAP]':
        return False
    else:
        return True

# 生成变异数据
def genVariant(variant_dic, seq_1, seq_2):
    
    raw_seq_1 = seq_1.copy()

    for variant, right in zip(seq_1, seq_2):
        
        if isVariant(variant, right):
            if right in variant_dic.keys():
                a = random.choice(variant_dic[right])
                while a == variant:
                    a = random.choice(variant_dic[right])
                    print(a)

                seq_1[seq_1.index(variant)] = a
    if seq_1 == raw_seq_1:
        return 0, 0
    else:
        return seq_1, seq_2


variantNW = VariantNW()
# 读取数据
data = pd.read_csv("../Data/Dataset/train_data.csv", index_col=0 ).reset_index(drop=True)
with open("../Data/variant_dic_remove_re.json", "r", encoding='UTF-8') as f:
    variant_dic = json.load(f)


raw_data = []
right_data = []
for row in data.iterrows():
    seq1 = row[1][0]
    seq2 = row[1][1]
    variantNW.set_seqs(seq1, seq2)
    variantNW.propagate()
    aligned_seq1, aligned_seq2 = variantNW.traceback()
    seq1,seq2 = genVariant(variant_dic, aligned_seq1, aligned_seq2)

    if seq1 == 0:
        continue
    else:
        try:
            seq1 = "".join(seq1).replace("[GAP]","")
            seq2 = "".join(seq2).replace("[GAP]","")
        except:
            pass
        if row[0] % 500 == 499:
            print("run")
        raw_data.append(seq1)
        right_data.append(seq2)
        # print(seq2)

dataframe = pd.DataFrame({"raw_data":raw_data, "right_data":right_data})
dataframe

In [ ]:
dataframe.to_csv("../Data/Dataset/train_data_supply_0417.csv")

In [ ]:
import sys
sys.path.append("..")
from Model_parts import RNNSearch,ConvS2S, TranslationModel
from Utils.config import Config
config = Config()
r = RNNSearch(config)
c = ConvS2S(config)
t= TranslationModel(src_vocab_size=config.source_vocab_size,
                                         tgt_vocab_size=config.target_vocab_size,
                                         d_model=config.d_model,
                                         nhead=config.num_head,
                                         num_encoder_layers=config.num_encoder_layers,
                                         num_decoder_layers=config.num_decoder_layers,
                                         dim_feedforward=config.dim_feedforward,
                                         dropout=config.dropout)
def get_model_size(model):
	para_num = sum([p.numel() for p in model.parameters()])
	# para_size: 参数个数 * 每个4字节(float32) / 1024 / 1024，单位为 MB
	para_size = para_num * 4 / 1024 / 1024
	return para_size

get_model_size(c)